In [ ]:
import cv2
import numpy as np
import face_recognition
import os

In [ ]:
path = './images/Dataset'
images=[]
classNames=[]
mylist=os.listdir(path)[1:]
print(mylist)

for cls in mylist:
    curImg=cv2.imread(f'{path}/{cls}')
    images.append(curImg)
    classNames.append(os.path.splitext(cls)[0])
    
print(classNames)

In [ ]:
def findEncodings(images):
    encodeList=[]
    for img in images:
        img=cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
        encode=face_recognition.face_encodings(img)[0]
        encodeList.append(encode)
    return encodeList

encodeListKnown=findEncodings(images)
print("Encoding complete!")

In [ ]:
cap=cv2.VideoCapture(0)

while True:
    res,img=cap.read()
    imgSmall=cv2.resize(img,(0,0),None,0.25,0.25)
    imgRGB=cv2.cvtColor(imgSmall,cv2.COLOR_BGR2RGB)
    
    facesCurFrame= face_recognition.face_locations(imgRGB)
    encodeCurFrame=face_recognition.face_encodings(imgRGB,facesCurFrame)
    
    for encodeFace,FaceLoc in zip(encodeCurFrame,facesCurFrame):
        matches=face_recognition.compare_faces(encodeListKnown,encodeFace)
        faceDis=face_recognition.face_distance(encodeListKnown,encodeFace)
        print(faceDis)
        name='UNKNOWN'
        matchIndex=np.argmin(faceDis)
        
        if matches[matchIndex]:
            print(f'index:{matchIndex}')
            name=classNames[matchIndex].upper()
            print(name)
        y1,x2,y2,x1=FaceLoc
        y1,x2,y2,x1=y1*4,x2*4,y2*4,x1*4
        cv2.rectangle(img,(x1,y1),(x2,y2),(0,255,0),2)
        cv2.rectangle(img,(x1,y2-35),(x2,y2),(0,255,0),cv2.FILLED)
        cv2.putText(img,name,(x1+6,y2-6),cv2.FONT_HERSHEY_COMPLEX,1,(255,255,255),2)
  
    cv2.imshow('frame',img)
    if cv2.waitKey(1) & 0xFF==ord('q'):
        break
    
    
cv2.destroyAllWindows()
cap.release()